In [4]:
import os
import numpy as np
import pandas as pd
import shutil
import cv2

In [14]:
base_dir = '/home/vuk/Documents/ML_Data/CDA1'
csv_loc = '/home/vuk/Documents/ML_Data/CDA1/HAM10000_metadata.csv'
ham_dir ='/home/vuk/Documents/ML_Data/CDA1/HAM10000'
test_dataset_dir = '/home/vuk/Melanoma-Detector/model/test_dataset'

In [21]:
!ls -l /home/vuk/Documents/ML_Data/CDA1/HAM10000

ls: cannot access '/home/vuk/Documents/ML_Data/CDA1/HAM1000': No such file or directory


In [28]:
for subfolder in ['benign','malignant']:
    subfolder_path = os.path.join(test_dataset_dir, subfolder)
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)
    else:
        print(f"Directory '{subfolder_path}' already exists.")

Directory '/home/vuk/Melanoma-Detector/model/test_dataset/benign' already exists.
Directory '/home/vuk/Melanoma-Detector/model/test_dataset/malignant' already exists.


In [36]:
!ls -l /home/vuk/Documents/ML_Data/CDA1/HAM10000 | grep 'ISIC_0027419'

-rw-rw-r-- 1 vuk vuk 311921 okt  6  2019 ISIC_0027419.jpg


In [43]:
with open(csv_loc) as file:
    # Skip the header line
    next(file)
    
    # Counter to track the number of images moved
    num_images_moved = 0
    
    # Iterate through each line in the file
    for line in file:
        # Split the line by comma
        split_line = line.split(',')
        
        # Extract the image file and tumor type
        img_file = split_line[1].strip()
        tumor_type = split_line[2].strip()

        # Determine the class directory based on the tumor type
        class_dir = 'benign' if tumor_type not in ['bcc', 'mel'] else 'malignant'
        
        # Construct the source file path
        source_file_path = os.path.join(ham_dir, img_file + '.jpg')
        
        # Construct the destination directory path
        destination_dir = os.path.join(test_dataset_dir, class_dir)
        
        # Create the destination directory if it doesn't exist
        if not os.path.exists(destination_dir):
            os.makedirs(destination_dir)
    
        # Construct the destination file path
        destination_file_path = os.path.join(destination_dir, img_file + '.jpg')
        
        # Move the image to the destination directory only if the filename is not "ISIC_0027419"
        if img_file != "ISIC_0027419":
            shutil.move(source_file_path, destination_file_path)
            num_images_moved += 1
        
        # Break the loop if 200 images have been moved
        if num_images_moved >= 200:
            break


In [ ]:
def pre_processing(img):
    image = cv2.imread(img_path)
    image_blurred = cv2.GaussianBlur(image, (5, 5), 1)
    lab_image = cv2.cvtColor(image_blurred, cv2.COLOR_BGR2LAB)
    gray_image = cv2.cvtColor(image_blurred, cv2.COLOR_BGR2GRAY)
    
    # To find upper threshold, we need to apply Otsu's thresholding
    upper_threshold, thresh_image = cv2.threshold(
        gray_image, thresh=0, maxval=255, type=cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )
    # Calculate lower threshold
    lower_threshold = 0.5 * upper_threshold
    
    # Apply canny edge detection
    canny = cv2.Canny(gray_image, lower_threshold, upper_threshold)
    # Finding the non-zero points of canny
    pts = np.argwhere(canny > 0)
    
    # Finding the min and max points
    y1, x1 = pts.min(axis=0)
    y2, x2 = pts.max(axis=0)
    
    # Crop ROI from the givn image
    output_image = image[y1:y2, x1:x2]

    return output_image

In [ ]:
def binary_mask(image):
    IL, Ia, Ib = cv2.split(img)
    avg_IL = np.mean(IL)
    avg_Ia = np.mean(Ia)
    avg_Ib = np.mean(Ib)

    print(avg_IL,avg_Ia,avg_Ib)
    
    IThL = np.where(IL >= avg_IL, 1, 0)
    ITha = np.where(Ia >= avg_Ia, 1, 0)
    IThb = np.where(Ib >= avg_Ib, 1, 0)

    Ibin = cv2.bitwise_and(IThL, ITha)
    Ibin = cv2.bitwise_and(Ibin, IThb)
    Ibin_uint8 = Ibin.astype(np.uint8)
    return Ibin

In [ ]:
def save_images(img, imgBin, destination_folder):
    cv2.imwrite(os.path.join(destination_folder, 'roi_' + img + '.jpg'), roi_image)
    cv2.imwrite(os.path.join(destination_folder, 'binary_' + imgBin + '.jpg'), binary_mask)

In [ ]:
for image in os.path('test_dataset'):
    img = output_image(image)
    imgBin = binary_mask(image)
    save_images(img,imgBin, 'test_dataset')
    
    